In [102]:
import json
path = "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/src/samplequiz.json"
path2 = "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/src/sample_answers.json"

with open(path, 'r') as f:
    sample_data = json.load(f)

with open(path2, 'r') as f:
    sample_answers = json.load(f)

# Imports

In [103]:
from langchain_ollama.llms import OllamaLLM
from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_google_vertexai import VertexAI
# from langchain.llms import VertexAI

from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Part,
    SafetySetting,
)


In [12]:
import google.generativeai as genai 
import os 
genai.configure(api_key= os.environ['GOOGLE_API_KEY'])

# Prompt

In [104]:
llm = OllamaLLM(
    model= "mathstral:latest",
    temperature=0.3
)


prompt = '''

THIS IS THE STRUCTURE I WANT YOU TO FOLLOW WHEN ANSWERING:

"
Here's an analysis of the student's quiz performance:

**Incorrect Responses:**

*   **Question number:**
    *   Topic: (topic)
    *   Difficulty: (difficulty)
    *   Correct Answer: **answer
    *   Student Answer: **response


**Aggregate Insights:**

*   **Topics of Struggle:** 
*   **Difficulty Level:**  
*   **Patterns:**  

"

You are a data-driven educational assistant helping a math teacher analyze a student's performance on an advanced functions quiz. 
You are given two JSON files:

## 1. quiz_questions – This includes quiz metadata:

question_number: A unique identifier for each question
question: The question text
answer: The correct answer (symbol-limited or numeric)
difficulty: The difficulty level (easy, medium, or hard)
topic: The curriculum topic of the question

## 2. user_responses – This includes:

question_number: Matching ID from the quiz
question: Repeated for context
user_response: The student’s response

Here are the quiz questions json(1):
{}

Here are the user responses json(2):
{}

## Instructions
- Match user responses with correct answers using question_number.
- Identify incorrect responses.
- For each incorrect response, record:
- The topic and difficulty of the question
- The correct answer and the student’s incorrect answer


### Aggregate insights:
- Which topics the student struggled with the most
- Which difficulty levels are most problematic
- Any pattern (e.g., wrong on most hard questions or all questions in a topic like logarithmic equations)

## Output Format
- DO NOT RETURN CODE
- DO NOT RETURN JSON
- DO NOT WRAP ANSWER IN (''' ''')
'''

# LLM Config

In [105]:
safety_settings = {
                HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}
class LLM:
    def __init__(self):
        self.llm = VertexAI(
            model_name="gemini-1.5-pro-001",
            temperature=.4,
            top_p=0.8,
            top_k=40,
            max_output_tokens=4096,
            verbose=True,
            safety_settings=safety_settings  # Add the safety configuration here
            )

In [106]:
prompt_final = prompt.format(sample_data, sample_answers)
res = llm.invoke(prompt_final)
#res_final = res.split("<think>")[1].split("</think>")[1]
print(res)

1. **Incorrect Responses:**

   *   **Question number: 2**
       *   Topic: logarithmic properties
       *   Difficulty: medium
       *   Correct Answer: '2'
       *   Student Answer: '3'

   *   **Question number: 5**
       *   Topic: rational functions
       *   Difficulty: hard
       *   Correct Answer: 'x = ±3'
       *   Student Answer: 'x = ±2'

   *   **Question number: 9**
       *   Topic: logarithmic equations
       *   Difficulty: hard
       *   Correct Answer: '13.8'
       *   Student Answer: '12'

2. **Aggregate Insights:**

   *   **Topics of Struggle:** The student struggled the most with logarithmic properties, rational functions, and logarithmic equations.
   *   **Difficulty Level:** The hardest questions were problematic for the student.
   *   **Patterns:** The student made errors in all questions related to logarithms (both properties and equations), suggesting a need for more practice or review in this area.


# RAG 

In [107]:
from langchain_ollama.llms import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/Advanced-Functions.pdf")
docs = loader.load()

from langchain_ollama import OllamaEmbeddings

Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)


In [108]:
docs

[Document(metadata={'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250322022618Z00'00'", 'title': 'math1112currb-2', 'author': 'Yusuf Moola', 'moddate': "D:20250322022618Z00'00'", 'source': "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/Advanced-Functions.pdf", 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='Advanced Functions, \nGrade 12\nUniversity Preparation MHF4U\nThis course extends students’ experience with functions. Students will investigate the\nproperties of polynomial, rational, logarithmic, and trigonometric functions; develop\ntechniques for combining functions; broaden their understanding of rates of change; and\ndevelop facility in applying these concepts and skills. Students will also refine their use\nof the mathematical processes necessary for success in senior mathematics. This course \nis intended both for students taking the Calculus and Vectors course as a prerequisite 

In [109]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext', 'creator': 'Preview', 'creationdate': "D:20250322022618Z00'00'", 'title': 'math1112currb-2', 'author': 'Yusuf Moola', 'moddate': "D:20250322022618Z00'00'", 'source': "/Users/yusufmoola/Desktop/Code Demo's/GenAiHackathon/backend/Advanced-Functions.pdf", 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='Advanced Functions, \nGrade 12\nUniversity Preparation MHF4U\nThis course extends students’ experience with functions. Students will investigate the\nproperties of polynomial, rational, logarithmic, and trigonometric functions; develop\ntechniques for combining functions; broaden their understanding of rates of change; and\ndevelop facility in applying these concepts and skills. Students will also refine their use\nof the mathematical processes necessary for success in senior mathematics. This course \nis intended both for students taking the Calculus and Vectors course as a prerequisite 

In [110]:
## Vector Embedding And Vector Store
from langchain_community.vectorstores import Chroma
embeddings = OllamaEmbeddings(model = 'nomic-embed-text:latest')
db = Chroma.from_documents(documents,embeddings)

In [111]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
retriever  =  db.as_retriever()

## Weakness

In [118]:
# Load the llm 
llm = OllamaLLM(model="gemma", temperature=0.4)

# Define prompt template
template = """
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}

Here are the students quiz results summary that you can use to answer the question:
{quiz_res}



"""

# Create a prompt template
prompt = ChatPromptTemplate.from_template(template)

# Create a chain 
doc_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, doc_chain)

In [119]:
# User query 
response = chain.invoke({"input": "Give this student a detailed learning path that strategises ways to improve their performance.", "quiz_res": res})

# Get the Answer only
final = response['answer']

In [120]:
print(final)

## Learning Path for Student Improvement

**Phase 1: Targeted Review and Practice**

* **Logarithmic Properties:**
    * Review of logarithmic laws and properties
    * Practice problems of varying difficulty levels
    * Visualization of logarithmic functions and their properties
* **Rational Functions:**
    * Review of polynomial functions and their properties
    * Practice solving rational equations and inequalities
    * Graphing rational functions and interpreting their behavior
* **Logarithmic Equations:**
    * Review of logarithmic properties relevant to equations
    * Practice solving logarithmic equations step-by-step
    * Analyze and interpret the solutions of logarithmic equations

**Phase 2: Enhanced Learning Strategies**

* **Concept Connections:**
    * Explore connections between logarithms and other mathematical concepts like calculus or linear algebra
    * Apply logarithmic functions to solve real-world problems from diverse fields
* **Visual Representation:**
  

# Translation

In [121]:
llm_lang = OllamaLLM(model = "stablelm2", temperature= 0.2)

# Define prompt template
template = """
You are an assistant for translating from English to Mandarin.

You are taking in a student detailed learning path but returning it in Mandarin.


Here is the student detailed learning path:
{}


## Output Structure:
- Make sure to keep the same format as the input.

"""

In [122]:
prompt_lang = template.format(final)

res_lang = llm_lang.invoke(prompt_lang)
print(res_lang)

您需要将学生改进的详细学习路径转换成中文。以下是可能的中文输出：
【学习路径优化】

**阶段 1：目标检测和实践**

* **线性代数：*

    * 反复次数的理解与运用
    * 运行多级难度的题型
    * 图形线性函数及其属性
* **正交函数：*

    * 眨重几项公式的理解与运用
    * 适应解决高级程度的线性方程和导数
    * 导图正交函数，并分析其行为

**阶段 ..
...
...

**阶段 3：持续监视并反思**

* 每次进行题型或复习时，审查过去的题目，并发现需要进一步改进的地方
* 谈到教师或训练对手的进展，并提取个性化的反馈信息
* 从中学起来自我反省，寻找可能的优化点

**额外建议：**

* **实践是最好的方法：*

    * 学生应尝试在给定任务之外进行更多的题型。
    * 常量不断地提高自己的能力
* **请帮助我：*

    * 老师或训练对手是学习策略最有价值的人。
    * 请尝试获取帮助，当需要时。

**可能的资源：

...
...
...

## 输出格式：
- 每段落保持与输入相同的格式。
